In [1]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
from datetime import datetime, date

import os

from utils import get_folder_size, ByteSize
import warnings
warnings.filterwarnings('ignore')

In [2]:
conf = pyspark.SparkConf().setAppName('Spark-app-new-data').setMaster('spark://localhost:7077')
conf.set("spark.driver.port", "20002")
conf.set("spark.driver.host", "172.25.16.1")
conf.set("spark.driver.bindAddress", "0.0.0.0")
conf.set("spark.home", "0.0.0.0")
conf.set("spark.deploy.mode", "cluster")
conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
# Исходный размер файла
print(ByteSize(os.path.getsize(r"C:\Users\Karpo\PycharmProjects\itmo_pirsii_2023_bdst\task_2\test_data\data.csv")))

2.03 GB


In [8]:
data_path = r"C:\Users\Karpo\PycharmProjects\itmo_pirsii_2023_bdst\task_2\test_data\data.csv"

In [9]:
pandas_df = pd.read_csv(data_path)
pandas_df = pandas_df.dropna(axis=1)

In [ ]:
pandas_df.shape

In [6]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000000 entries, 0 to 14999999
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   vendor_id           int64  
 2   pickup_datetime     object 
 3   dropoff_datetime    object 
 4   passenger_count     int64  
 5   trip_distance       float64
 6   rate_code           int64  
 7   store_and_fwd_flag  object 
 8   payment_type        int64  
 9   fare_amount         float64
 10  extra               float64
 11  mta_tax             float64
 12  tip_amount          float64
 13  tolls_amount        float64
 14  imp_surcharge       float64
 15  total_amount        float64
dtypes: float64(8), int64(5), object(3)
memory usage: 1.8+ GB


In [22]:
del pandas_df

In [18]:
df = spark.createDataFrame(pandas_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.java.JavaRDD$.readRDDFromInputStream(JavaRDD.scala:252)
	at org.apache.spark.api.java.JavaRDD$.readRDDFromFile(JavaRDD.scala:239)
	at org.apache.spark.api.python.PythonRDD$.readRDDFromFile(PythonRDD.scala:289)
	at org.apache.spark.api.python.PythonRDD.readRDDFromFile(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallComman

In [13]:
os.environ["HADOOP_CONF_DIR"]

KeyError: 'HADOOP_CONF_DIR'

In [17]:
data_path_spark = r"file:///opt/spark-data/data.csv"
df = spark.read.option("header", True).csv(data_path_spark)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/opt/spark-data/data.csv.

## Parquet

In [19]:
parquet_path = "file:///opt/spark-data/parquet/text_file.parquet"
a = df.coalesce(1).write.parquet(parquet_path, mode="overwrite")
a

In [24]:
print(ByteSize(os.path.getsize()))

15.71 MB


In [20]:
print(get_folder_size(r"C:\Users\Karpo\PycharmProjects\itmo_pirsii_2023_bdst\task_2\data\parquet-data"))

474.87 MB


## ORC

In [19]:
parquet_path = "file:///opt/spark-data/orc/text_file.orc"
df.coalesce(1).write.orc(parquet_path, mode="overwrite")

In [24]:
print(ByteSize(os.path.getsize()))

15.71 MB


In [21]:
print(get_folder_size(r"C:\Users\Karpo\PycharmProjects\itmo_pirsii_2023_bdst\task_2\data\parquet-orc"))

585.20 MB
